## 1. Load dataset

In [1]:
from os.path import join
import pandas as pd


def relabeler(label):
    if label == 'entailment':
        return 0
    if label == 'neutral':
        return 1
    if label == 'contradiction':
        return 2
    else:
        None


def _load(fname, genre=None, dire='data/data135628/'):
    df = pd.read_csv(join(dire, fname))
    if genre:
        genres = ['slate', 'government', 'telephone',
                  'travel', 'fiction']

        if isinstance(genre, str):
            genre = [genre]
        elif isinstance(genre, (list, tuple)):
            assert all(isinstance(g, str) for g in genre)
        else:
            raise TypeError("genre must be a str or a list of str")

        assert all(g in genres for g in genre), f'genre must be in {genres}'
        df = df[df.isin(genre)['genre']]

    df = df.fillna(" ")
    if 'test' in fname:
        df = df[['sentence1', 'sentence2']]
    else:
        df['gold_label'] = df['gold_label'].apply(relabeler)
        df.dropna(subset=['sentence1', 'sentence2', 'gold_label'], inplace=True)
        df = df[['sentence1', 'sentence2', 'gold_label']]

    out = []
    for row in df.values:
        out.append(row.tolist())
    return out


def load_dataset(fname, genre=None, dire='data/data135628/'):
    if isinstance(fname, str):
        return _load(fname, genre, dire)

    elif isinstance(fname, (list, tuple)):
        return [_load(f, genre, dire) for f in fname]

    else:
        raise TypeError("fname must be a str or a list!")

In [2]:
train_set, dev1_set, dev2_set = load_dataset(['train.csv', 'dev_matched.csv', 'dev_mismatched.csv'])
dev_set = dev1_set + dev2_set

test1_set, test2_set = load_dataset(['test_matched.csv', 'test_mismatched.csv'])
print("Train set size:", len(train_set))
print("Train set examples:", train_set[:2])

print("\nDev set size:", len(dev_set))
print("nDev set examples:", dev_set[:2])

print("\nTest1 set size:", len(test1_set))
print("nTest2 set examples:", test1_set[:2])

print("\nTest2 set size:", len(test2_set))
print("nTest2 set examples:", test2_set[:2])

Train set size: 392702
Train set examples: [['Conceptually cream skimming has two basic dimensions - product and geography.', 'Product and geography are what make cream skimming work. ', 1], ['you know during the season and i guess at at your level uh you lose them to the next level if if they decide to recall the the parent team the Braves decide to call to recall a guy from triple A then a double A guy goes up to replace him and a single A guy goes up to replace him', 'You lose the things to the following level if the people recall.', 0]]

Dev set size: 19647
nDev set examples: [['The new rights are nice enough', 'Everyone really likes the newest benefits ', 1.0], ['This site includes a list of all award winners and a searchable database of Government Executive articles.', 'The Government Executive articles housed on the website are not able to be searched.', 2.0]]

Test1 set size: 9796
nTest2 set examples: [['That which binds together Chinese.', 'This is a shared value among Chinese

## 2. Transform text

In [3]:
from paddlenlp.datasets import MapDataset
from paddle.io import BatchSampler, DataLoader
from paddlenlp.data import Pad, Stack, Tuple
from paddlenlp.transformers import BertModel as SeqClfModel
from paddlenlp.transformers import BertTokenizer as PTMTokenizer


MODEL_NAME = "bert-large-uncased"
tokenizer = PTMTokenizer.from_pretrained(MODEL_NAME)


def example_converter(example, max_seq_length, tokenizer):
    text_a, text_b, label = example
    encoded = tokenizer(text=text_a, text_pair=text_b, max_seq_len=max_seq_length)
    input_ids = encoded["input_ids"]
    token_type_ids = encoded["token_type_ids"]
    return input_ids, token_type_ids, label


def get_trans_fn(max_seq_length=128, tokenizer=tokenizer):
    return lambda ex: example_converter(ex, max_seq_length, tokenizer)


batchify_fn = lambda samples, fn=Tuple(
    Pad(axis=0, pad_val=tokenizer.pad_token_id),
    Pad(axis=0, pad_val=tokenizer.pad_token_type_id),
    Stack(dtype="int64")
    ): fn(samples)


def create_dataloader(dataset,
                      trans_fn,
                      batchify_fn,
                      test=False,
                      batch_size=128,
                      shuffle=True,
                      sampler=BatchSampler):

    if test:
        dataset = [d + [0] for d in dataset]

    if not isinstance(dataset, MapDataset):
        dataset = MapDataset(dataset)

    dataset.map(trans_fn)
    batch_sampler = sampler(dataset,
                            shuffle=shuffle,
                            batch_size=batch_size)

    dataloder = DataLoader(dataset,
                           batch_sampler=batch_sampler,
                           collate_fn=batchify_fn)

    return dataloder

[2022-04-02 01:35:53,442] [    INFO]

 - Downloading https://paddle-hapi.bj.bcebos.com/models/bert/bert-large-uncased-vocab.txt and saved to /home/aistudio/.paddlenlp/models/bert-large-uncased

[2022-04-02 01:35:53,445] [    INFO]

 - Downloading bert-large-uncased-vocab.txt from https://paddle-hapi.bj.bcebos.com/models/bert/bert-large-uncased-vocab.txt

  0%|          | 0/227 [00:00<?, ?it/s]

100%|██████████| 227/227 [00:00<00:00, 36966.42it/s]

In [4]:
max_seq_length = 128; batch_size = 64
trans_fn = get_trans_fn(max_seq_length)
train_loader = create_dataloader(train_set, trans_fn, batchify_fn, batch_size=batch_size)
dev_loader = create_dataloader(dev_set, trans_fn, batchify_fn, batch_size=batch_size)
test1_loader = create_dataloader(test1_set, trans_fn, batchify_fn, test=True, shuffle=False, batch_size=batch_size)
test2_loader = create_dataloader(test2_set, trans_fn, batchify_fn, test=True, shuffle=False, batch_size=batch_size)

## 3. Model building

In [5]:
from paddle import nn
import paddle


class PTM(nn.Layer):

    def __init__(self, pretrained_model, dropout=0.1, num_class=3):
        super().__init__()

        self.ptm = pretrained_model
        ptm_out_dim = self.ptm.config["hidden_size"]
        self.dropout = nn.Dropout(dropout)
        self.fc1 = nn.Linear(ptm_out_dim, ptm_out_dim // 2)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(ptm_out_dim // 2, num_class)

    def encoder(self, input_ids, token_type_ids):
        _, embd = self.ptm(input_ids, token_type_ids)
        embd = self.dropout(embd)
        return embd

    def forward(self, input_ids, token_type_ids):
        embd = self.encoder(input_ids, token_type_ids)
        hidden = self.relu(self.fc1(embd))
        logits = self.fc2(hidden)
        return logits

In [6]:
from paddlenlp.transformers import LinearDecayWithWarmup

epoch = 10
weight_decay = 0.0
warmup_proportion = 0.0
lr_scheduler = LinearDecayWithWarmup(5e-6, len(train_loader) * epoch,
                                         warmup_proportion)

def get_model(model):
    decay_params = [
        p.name for n, p in model.named_parameters()
        if not any(nd in n for nd in ["bias", "norm"])
    ]
    optimizer = paddle.optimizer.AdamW(
    parameters=model.parameters(),
    learning_rate=lr_scheduler,
    weight_decay=weight_decay,
    apply_decay_param_fun=lambda x: x in decay_params)

    criterion = paddle.nn.CrossEntropyLoss()

    model = paddle.Model(model)
    metric = paddle.metric.Accuracy()
    model.prepare(optimizer, criterion, metric)
    return model

In [7]:
model = SeqClfModel.from_pretrained(MODEL_NAME)
model = PTM(model)
model = get_model(model)

[2022-04-02 01:35:53,676] [    INFO]

 - Downloading https://paddlenlp.bj.bcebos.com/models/transformers/bert-large-uncased.pdparams and saved to /home/aistudio/.paddlenlp/models/bert-large-uncased

[2022-04-02 01:35:53,678] [    INFO]

 - Downloading bert-large-uncased.pdparams from https://paddlenlp.bj.bcebos.com/models/transformers/bert-large-uncased.pdparams

  0%|          | 0/2158322 [00:00<?, ?it/s]

...

100%|█████████▉| 2152635/2158322 [00:32<00:00, 75407.85it/s]

100%|██████████| 2158322/2158322 [00:32<00:00, 66423.38it/s]

W0402 01:36:26.275269   255 device_context.cc:447] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 10.1, Runtime API Version: 10.1
W0402 01:36:26.280350   255 device_context.cc:465] device: 0, cuDNN Version: 7.6.


[2022-04-02 01:36:49,006] [    INFO]

 - Weights from pretrained model not used in BertModel: ['cls.predictions.decoder_weight', 'cls.predictions.decoder_bias', 'cls.predictions.transform.weight', 'cls.predictions.transform.bias', 'cls.predictions.layer_norm.weight', 'cls.predictions.layer_norm.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']

## 4. Model training

In [8]:
from paddle.callbacks import EarlyStopping


earlystop = EarlyStopping(patience=3)
model.fit(train_loader, dev_loader, epochs=epoch, verbose=2, log_freq=100, callbacks=[earlystop])

The loss value printed in the log is the current step, and the metric is the average value of previous steps.

Epoch 1/10

step  100/6136 - loss: 1.1554 - acc: 0.3495 - 1s/step


step  200/6136 - loss: 0.9542 - acc: 0.4062 - 1s/step


step  300/6136 - loss: 1.0030 - acc: 0.4831 - 1s/step


...

step 200/307 - loss: 0.7084 - acc: 0.8602 - 405ms/step


step 300/307 - loss: 0.6246 - acc: 0.8591 - 396ms/step


step 307/307 - loss: 0.4375 - acc: 0.8588 - 395ms/step


Eval samples: 19647

Epoch 6: Early stopping.

## 5. Prediction

In [9]:
import paddle.nn.functional as F
from tqdm import tqdm


def predict(test_loader, id_inpath, out):
    predictions = []
    logits = model.predict(test_loader)

    for batch in tqdm(logits[0]):
        batch = paddle.to_tensor(batch)
        probs = F.softmax(batch, axis=1)
        preds = paddle.argmax(probs, axis=1).numpy().tolist()
        predictions.extend(preds)

    df = pd.read_csv(id_inpath)
    ids = df['pairID'].tolist()
    label_map = {0: 'entailment', 1: 'neutral', 2: 'contradiction'}
    results = [[idx, label_map[p]] for idx, p in zip(ids, predictions)]
    columns = ['pairID', 'gold_label']
    pd.DataFrame(results, columns=columns).to_csv(out, index=False)
    print(f"{out} has been saved!")

In [10]:
predict(test1_loader, 'data/data135628/test_matched.csv', 'test_matched_preds.csv')

Predict begin...

step   2/154 [..............................]

 - ETA: 1:04 - 423ms/step



step   4/154 [..............................]

...

 - 391ms/step          


Predict samples: 9796

  0%|          | 0/154 [00:00<?, ?it/s]

100%|██████████| 154/154 [00:00<00:00, 9159.69it/s]

test_matched_preds.csv has been saved!

In [11]:
predict(test2_loader, 'data/data135628/test_mismatched.csv', 'test_mismatched_preds.csv')

Predict begin...

step   2/154 [..............................]

 - ETA: 1:14 - 490ms/step



step   4/154 [..............................]

...

 - 387ms/step          


Predict samples: 9847

  0%|          | 0/154 [00:00<?, ?it/s]

100%|██████████| 154/154 [00:00<00:00, 9028.71it/s]

test_mismatched_preds.csv has been saved!